# 환경설정

In [ ]:
# !pip install scikit-learn==1.3.0 -q
# !pip install torch==2.0.1 -q
# !pip install torchvision==0.15.2 -q
# !pip install torchtext==0.15.2 -q

In [1]:
!pip install scikit-learn
!pip install torch
!pip install torchvision
!pip install torchtext

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import numpy as np # 기본적인 연산을 위한 라이브러리
import matplotlib.pyplot as plt # 그림이나 그래프를 그리기 위한 라이브러리
from tqdm.notebook import tqdm # 상태 바를 나타내기 위한 라이브러리
import pandas as pd # 데이터프레임을 조작하기 위한 라이브러리

import torch # PyTorch 라이브러리
import torch.nn as nn # 모델 구성을 위한 라이브러리
import torch.optim as optim # optimizer 설정을 위한 라이브러리
from torch.utils.data import Dataset, DataLoader # 데이터셋 설정을 위한 라이브러리

from torchtext.data import get_tokenizer # torch에서 tokenizer를 얻기 위한 라이브러리
import torchtext # torch에서 text를 더 잘 처리하기 위한 라이브러리

from sklearn.metrics import accuracy_score # 성능지표 측정
from sklearn.model_selection import train_test_split # train-validation-test set 나누는 라이브러리

import re # text 전처리를 위한 라이브러리

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [3]:
# seed 고정
import random
import torch.backends.cudnn as cudnn

def random_seed(seed_num):
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    torch.cuda.manual_seed_all(seed_num)
    np.random.seed(seed_num)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seed_num)

random_seed(42)

In [4]:
# device = 'cuda:0'
device = torch.device('cpu')

## Dataset

* 데이터셋: <a href='https://www.kaggle.com/datasets/dorianlazar/medium-articles-dataset'>Medium Dataset</a>
* 데이터셋 개요: "Towards Data Science", "UX Collective", "The Startup", "The Writing Cooperative", "Data Driven Investor", "Better Humans", "Better Marketing" 의 7개의 주제를 가지는 publication 에 대해서 크롤링을 한 데이터입니다. 원본 데이터는 총 6,508개의 블로그 이미지와 메타 데이터(.csv)로 구성됩니다. 실습에서는 메타데이터를 사용하여 CustomDataset을 구현합니다.
  * [How to collect ths dataset?](https://dorianlazar.medium.com/scraping-medium-with-python-beautiful-soup-3314f898bbf5)
- 메타 데이터 스키마: 메타 데이터는 총 **10**개의 column으로 구성됩니다.
  - id: 아이디
  - url: 포스팅 링크
  - title: 제목
  - subtitle: 부제목
  - image: 포스팅 이미지의 파일 이름
  - claps: 추천 수
  - reponses: 댓글 수
  - reading_time: 읽는데 걸리는 시간
  - publication: 주제 카테고리(e.g. Towards Data Science..)
  - date: 작성 날짜
- 데이터 셋 저작권: CC0: Public Domain

# 1.Custom Dataset 구축하기

## 1-1. 자연어 데이터의 전처리

text로 된 데이터를 어떻게 숫자 형식으로 바꾸고, 모델에 넣는 구조로 바꾸는지 실습

### Next word prediction
* 글의 일부가 주어졌을 때, 다음 단어를 예측 (next word prediction)하는 모델을 구축하는 것을 목표로 합니다.
* 예를 들어, "나는 학교를 가서 밥을 먹었다." 라는 문장이 주어진다고 해봅시다.

|input|label|
|------|---|
|나는|학교를|
|나는 학교를|가서|
|나는 학교를 가서|밥을|
|나는 학교를 가서 밥을|먹었다.|

* 이와 같이 데이터셋을 구축하고, DNN을 통해 다음 단어를 예측해봅니다.

* [Next word prediction](https://wikidocs.net/45101)

In [6]:
data_csv = pd.read_csv('medium_data.csv')
data_csv.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [7]:
print(data_csv.shape)

(6508, 10)


In [8]:
# 각각의 title만 추출합니다.
# 우리는 title의 첫 단어가 주어졌을 때, 다음 단어를 예측하는 것을 수행할 것입니다.
data = data_csv['title'].values

### 텍스트 데이터 전처리하기
* 해당 데이터셋은 크롤링(인터넷에 있는 정보를 수집하는 기법)을 통해 구축되었기 때문에 no-break space가 종종 발생합니다. 이러한 no-break space를 제거하는 전처리를 진행합니다.
  * No-Break Space란? 웹 페이지나 문서 등에서 단어나 문장 사이의 공백이 있는 경우, 해당 공백이 줄 바꿈으로 인해 분리되지 않고 한 단어나 문장으로 인식되도록 하는데 사용되는 공백
  * 예시 (no-break-space 사용 X)
    ```
    Hello
    World~
    ```
    
    (no-break-space 사용)
    ```
    Hello,⎵world!
    ```
* no-break space를 제거하기 위해선 unicode 형식으로 제거를 해야합니다.
  * unicode란? 전 세계의 모든 문자와 기호를 일관성 있게 표현하기 위한 표준 문자 인코딩 체계

* `re` 라이브러리를 이용하여 텍스트 데이터를 쉽게 처리할 수 있습니다.

* <a href='https://www.compart.com/en/unicode'>unicode 검색 사이트</a>
* [re 라이브러리를 이용한 텍스트 데이터 사용법](https://velog.io/@hoegon02/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-12-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%A0%84%EC%B2%98%EB%A6%AC-%EC%A0%95%EA%B7%9C-%ED%91%9C%ED%98%84%EC%8B%9D-3qmtwryf)

In [9]:
def cleaning_text(text):
    cleaned_text = re.sub( r"[^a-zA-Z0-9.,@#!\s']+", "", text) # 특수문자 를 모두 지우는 작업을 수행합니다.
    cleaned_text = cleaned_text.replace(u'\xa0',u' ') # No-break space를 unicode 빈칸으로 변환
    cleaned_text = cleaned_text.replace('\u200a',' ') # unicode 빈칸을 빈칸으로 변환
    return cleaned_text

cleaned_data = list(map(cleaning_text, data)) # 모든 특수문자와 공백을 지움
print('Before preprocessing')
print(data[:5])
print('After preprocessing')
print(cleaned_data[:5])

Before preprocessing
['A Beginner’s Guide to Word Embedding with Gensim Word2Vec\xa0Model'
 'Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric'
 'How to Use ggplot2 in\xa0Python'
 'Databricks: How to Save Files in CSV on Your Local\xa0Computer'
 'A Step-by-Step Implementation of Gradient Descent and Backpropagation']
After preprocessing
['A Beginners Guide to Word Embedding with Gensim Word2Vec Model', 'Handson Graph Neural Networks with PyTorch  PyTorch Geometric', 'How to Use ggplot2 in Python', 'Databricks How to Save Files in CSV on Your Local Computer', 'A StepbyStep Implementation of Gradient Descent and Backpropagation']


### Tokenizer
* Tokenizer는 텍스트 데이터를 작은 단위로 분리해주는 도구입니다.
* 텍스트 데이터를 머신 러닝 모델에 입력으로 사용하거나 자연어 처리 작업을 수행할 때, 문장을 단어 또는 하위 단위(subword)로 분리하는 역할을 수행하기 위한 도구입니다.
  * 텍스트를 단어 또는 하위 단위로 분리 (토큰 분리): 텍스트를 띄어쓰기 단위로 나누거나, 보다 작은 단위로 분리합니다.
    * 예를 들어, "I love PyTorch"이라는 문장을 단어 단위로 분리하면 ["I", "love", "PyTorch"]과 같이 됩니다.
    * 하위 단위 토크나이저는 언어의 특성에 따라 단어를 더 작은 단위로 분리하여 처리할 수 있습니다. 예를 들어, "playing"이라는 단어를 "play"와 "ing"으로 분리하는 것입니다.

  * 토큰을 숫자로 매핑: 머신 러닝 모델은 텍스트를 숫자로 처리해야 합니다. 따라서 모델이 텍스트를 처리할 수 있게 단어나 하위 단위를 고유한 숫자 ID로 매핑하는 작업을 수행합니다.
    * 예를 들어, ["I", "love", "PyTorch"] 이라는 단어들이 있을 때, 이를 이용하여 {"I":0, "love":1, "PyTorch":2}와 같은 단어 사전을 만들고, 이를 통해 [0, 1, 2]로 변환합니다.
  * 특수 토큰 추가: 텍스트를 모델에 입력으로 사용할 때, 특별한 의미를 가진 토큰을 추가할 수 있습니다.
    * 예를 들어 문장의 시작(<sos> 토큰)과 끝(<eox> 토큰)을 나타내는데 사용되거나, 미리 정의된 사전에 없는 단어를 대체하는데 사용될 수 있습니다.
    
* 자연어 처리를 위한 라이브러리인 `torchtext.vocab.build_vocab_from_iterator`를 이용하여 위 과정을 모두 쉽게 처리할 수 있습니다.

* [torchtext getTokenizer](https://pytorch.org/text/stable/data_utils.html#get-tokenizer)
* [Vocab tokenize 설명](https://velog.io/@nkw011/nlp-vocab)

### build_vocab_from_iterator
`torchtext.vocab.build_vocab_from_iterator`는 iterator를 이용하여 Vocab 클래스(단어사전)를 만드는 함수입니다.
* 주요 parameter
  * iterator: 단어 사전을 만들 때 사용되는 iterator
  * min_freq: 단어 사전에 포함되기 위한 최소 빈도 수
* output
  * torchtext.vocab.Vocab 클래스를 반환합니다.
  * 이로써 Vocab class에 있는 함수들을 모두 사용할 수 있습니다.

* [build_vocab_from_iterator](https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator)
* [Vocab class의 함수들](https://pytorch.org/text/stable/vocab.html)

In [10]:
# 토크나이저를 통해 단어 단위의 토큰을 생성합니다.
'''
# get_tokenizer 함수: 지정된 방법(여기서는 "basic_english")에 따라 텍스트를 토큰화하는 함수 또는 객체를 반환.
# "basic_english"는 영어 문장을 단어 단위로 분리하는 기본적인 토크나이저를 의미
---------------------------------------------------------------------------------
# tokenizer로 불러온 토크나이저 객체를 사용하여 cleaned_data[0]에 있는 텍스트를 토큰화
# cleaned_data는 미리 처리된 텍스트 데이터의 리스트라고 가정할 수 있으며, [0]은 그 리스트의 첫 번째 요소를 의미
# 결과로 tokens 변수에는 텍스트가 단어 단위로 분리된 토큰들의 리스트가 저장
'''
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(cleaned_data[0])
print("Original text : ", cleaned_data[0])   # 원본 텍스트(토큰화하기 전의 텍스트)를 출력
print("Token: ", tokens)                     # 토큰화된 결과인 단어 리스트를 출력, 공백을 기준으로 만들어짐

Original text :  A Beginners Guide to Word Embedding with Gensim Word2Vec Model
Token:  ['a', 'beginners', 'guide', 'to', 'word', 'embedding', 'with', 'gensim', 'word2vec', 'model']


아래 코드는 주어진 텍스트 데이터에서 단어 사전을 생성하고, 패딩 용도로 사용될 '<pad>' 토큰을 사전에 추가하는 과정을 보여줍니다. 이를 통해 단어 사전에는 텍스트에 사용된 모든 단어와 특수 토큰이 포함되며, 각 단어는 고유한 인덱스를 가지게 됩니다.

In [15]:
# 단어 사전을 생성한 후, 시작과 끝 표시를 해줍니다.
'''
# torchtext.vocab.build_vocab_from_iterator 함수는 반복 가능한 객체(iterator)로부터 단어 사전을 생성
# map(tokenizer, cleaned_data)는 cleaned_data의 각 항목에 대해 tokenizer 함수를 적용하여 각 텍스트를 토큰화. 즉 cleaned_data의 각 요소(문장 등)를 단어 단위로 분리하여 반복 가능한 객체를 만듦
# vocab 변수에는 텍스트 데이터의 단어 사전이 생성
--------------------------------------------------------------------------------------------
# vocab.insert_token 함수는 단어 사전에 새로운 토큰을 삽입
# 여기서는 '<pad>'라는 특수 토큰을 단어 사전에 추가하며, 이 토큰의 인덱스를 0으로 설정
# <pad>' 토큰은 일반적으로 패딩(padding) 용도로 사용되며, 시퀀스의 길이를 맞추기 위해 사용됨
'''

vocab = torchtext.vocab.build_vocab_from_iterator(map(tokenizer, cleaned_data)) # 단어 사전을 생성합니다.
vocab.insert_token('<pad>', 0)

아래 코드는 단어 사전에서 각 인덱스에 해당하는 단어들을 리스트 형태로 가져와서, 그 리스트의 처음 10개 단어를 출력하는 과정을 보여줍니다. 이를 통해 단어 사전이 제대로 생성되었는지, 그리고 각 인덱스에 올바른 단어가 매핑되었는지 확인할 수 있습니다.

In [16]:
'''
id2token = vocab.get_itos()
    # vocab.get_itos()는 단어 사전 객체에서 인덱스를 문자열(단어)로 변환하는 함수
    # itos는 "index to string"의 약자로, 인덱스에서 문자열로 변환할 때 사용
    # 이 함수는 단어 사전의 각 인덱스에 해당하는 단어들을 리스트 형태로 반환
    # vocab 사전에서 인덱스 0에 해당하는 단어는 <pad>일 것이고, 인덱스 1에는 다른 단어가 올 것
    # id2token 변수에는 이러한 단어들이 리스트로 저장
id2token[:10]
    # id2token 리스트의 첫 10개 요소(단어)를 반환
'''
id2token = vocab.get_itos() # id to string
id2token[:10]

['<pad>', 'to', 'the', 'a', 'of', 'and', 'how', 'in', 'your', 'for']

 > 생성된 단어 사전에서 문자열(단어)을 인덱스로 변환하는 작업을 수행

 아래 코드는
 * 단어 사전에서 단어를 인덱스로 변환하는 딕셔너리를 생성하고,
 * 이를 인덱스 순서대로 정렬한 후,
 * 처음 6개의 단어와 해당 인덱스를 출력하는 과정을 보여줍니다.

이를 통해 사전에 포함된 단어들이 올바르게 인덱싱되었는지 확인할 수 있습니다.

In [17]:
'''
token2id = vocab.get_stoi()
    # vocab.get_stoi()는 단어 사전 객체에서 문자열(단어)을 인덱스로 변환하는 함수
    # stoi는 "string to index"의 약자로, 단어를 인덱스로 변환할 때 사용
    # 이 함수는 단어 사전의 각 단어에 해당하는 인덱스들을 딕셔너리 형태로 반환
    # token2id 변수에는 이러한 딕셔너리 저장

token2id = dict(sorted(token2id.items(), key=lambda item: item[1]))
    # token2id 딕셔너리를 인덱스 값을 기준으로 정렬
    # sorted(token2id.items(), key=lambda item: item[1])는 token2id 딕셔너리의 아이템을 인덱스 값(item[1])을 기준으로 정렬하여 리스트로 반환
    # dict() 함수를 사용하여 정렬된 리스트를 다시 딕셔너리로 변환
    # 결과적으로 token2id는 인덱스 값에 따라 정렬된 딕셔너리가 됨

for idx, (k,v) in enumerate(token2id.items()):
    # token2id 딕셔너리의 각 아이템(단어와 인덱스 쌍)을 반복
    # enumerate() 함수를 사용하여 반복 시 인덱스(idx)와 아이템(k, v)을 함께 가져옴

print(k, v)
    # 반복 중인 단어(k)와 그에 해당하는 인덱스(v)를 출력

if idx == 5:
    # 인덱스(idx)가 5인지 확인

break
    # 현재 반복 인덱스가 5이면 반복. 즉, 처음 6개의 단어와 인덱스를 출력한 후 반복을 멈춤.
'''
token2id = vocab.get_stoi() # string to id
token2id = dict(sorted(token2id.items(), key=lambda item: item[1]))
for idx, (k,v) in enumerate(token2id.items()):
    print(k,v)
    if idx == 5:
        break

<pad> 0
to 1
the 2
a 3
of 4
and 5


> 주어진 문장을 토큰화한 후, 각 토큰을 해당하는 단어 사전의 인덱스로 변환하는 과정을 보여줍니다.

아래 코드는
* 첫 번째 텍스트 데이터를 단어 단위로 토큰화한 후,
* 각 토큰을 단어 사전에서 해당하는 인덱스로 변환하여 인덱스 리스트를 반환
* 이를 통해 원본 텍스트 데이터를 숫자 인덱스 형태로 변환할 수 있습니다.

이 과정은 텍스트 데이터를 모델에 입력하기 위해 필수적인 전처리 단계입니다.

In [18]:
'''
tokenizer(cleaned_data[0])
    # cleaned_data[0]는 cleaned_data 리스트의 첫 번째 요소로, 텍스트 데이터 중 하나
    # tokenizer는 텍스트 데이터를 단어 단위로 분리하는 함수
    # 따라서, tokenizer(cleaned_data[0])는 첫 번째 텍스트 데이터를 단어 단위로 토큰화하여 토큰의 리스트를 반환

vocab.lookup_indices(...)
    # vocab.lookup_indices()는 단어 사전에서 주어진 토큰 리스트의 각 토큰을 해당 인덱스로 변환하는 함수
    # 여기서는 tokenizer(cleaned_data[0])를 통해 얻은 토큰 리스트를 인자로 전달하여, 각 토큰을 사전에 저장된 인덱스로 변환
    # 결과적으로, 이 함수는 토큰 리스트를 해당 인덱스 리스트로 변환하여 반환
'''
vocab.lookup_indices(tokenizer(cleaned_data[0])) # 문장을 토큰화 후 id로 변환합니다.

[3, 273, 66, 1, 467, 1582, 12, 2884, 8549, 99]

### 데이터 전처리

  
* input에 들어가는 단어 수가 모두 다르므로 이를 바로 모델에 넣기에는 어렵습니다. 이를 위해, \<pad\> (0)을 넣어서 길이를 맞춰주는 과정을 padding 이라고 합니다.
<!-- * label 값은 OneHotEncoding을 해야합니다.
  * torch.nn.functional.one_hot 함수를 이용하여 onehot encoding을 쉽게 할 수 있습니다.
  * OneHotEncoding이란? : 카테고리 형태의 데이터를 벡터로 변환하는 방법으로, 해당하는 카테고리에 해당하는 인덱스만 1이고 나머지는 모두 0인 이진 벡터로 표현하는 것을 의미합니다.
  * 왜 OneHotEncodingd을 해야할까? : multi-class(개, 고양이, 토끼 분류와 같은) 문제로 풀기 위함입니다.   -->
  

* [Padding 설명](https://wikidocs.net/83544)

> cleaned_data 리스트에 있는 모든 텍스트 데이터를 처리하여 각 텍스트의 부분 시퀀스를 생성한 후, 이를 seq 리스트에 추가하는 과정을 보여줍니다

In [22]:


'''
seq = []
    # seq는 최종 결과를 저장할 빈 리스트. 여기에는 모든 텍스트 데이터의 부분 시퀀스가 저장됩니다.

for i in cleaned_data:
    # cleaned_data 리스트의 각 텍스트 데이터를 순회. i는 현재 처리 중인 텍스트 데이터.

token_id = vocab.lookup_indices(tokenizer(i))
    # 현재 텍스트 데이터 i를 단어 단위로 토큰화하고, 각 토큰을 단어 사전에서 해당 인덱스로 변환.
    # tokenizer(i)는 텍스트 i를 단어 단위로 분리한 토큰 리스트를 반환.
    # vocab.lookup_indices(tokenizer(i))는 이 토큰 리스트를 단어 사전의 인덱스 리스트로 변환.
    # 결과적으로 token_id 변수에는 현재 텍스트 데이터의 각 단어가 인덱스로 변환된 리스트 저장.

for j in range(1, len(token_id)):
    # token_id 리스트의 각 인덱스를 순회. j는 현재 인덱스 위치.
    # range(1, len(token_id))는 1부터 token_id 리스트의 길이 - 1까지 반복. 이 범위는 부분 시퀀스를 생성하기 위한 범위.

sequence = token_id[:j+1]
    # token_id 리스트의 부분 시퀀스를 생성.
    # token_id[:j+1]는 token_id 리스트의 처음부터 j+1 위치까지의 부분 리스트를 반환.
    # 예를 들어, token_id가 [1, 2, 3, 4]이고 j가 2일 때, token_id[:j+1]는 [1, 2, 3]을 반환.

seq.append(sequence)
    # 생성된 부분 시퀀스를 seq 리스트에 추가.
    # 이 과정은 현재 텍스트 데이터의 모든 부분 시퀀스를 seq 리스트에 저장.
'''

seq = []
for i in cleaned_data:
    token_id = vocab.lookup_indices(tokenizer(i))
    for j in range(1, len(token_id)):
        sequence = token_id[:j+1]
        seq.append(sequence)

In [23]:
seq[:5]   # seq 리스트의 처음 5개의 요소를 출력

[[3, 273],
 [3, 273, 66],
 [3, 273, 66, 1],
 [3, 273, 66, 1, 467],
 [3, 273, 66, 1, 467, 1582]]

In [24]:
# seq 리스트에 저장된 부분 시퀀스들 중 가장 긴 시퀀스의 길이를 찾고 그 값을 출력
'''
# seq 리스트의 각 요소인 sublist의 길이(len(sublist))를 반복하면서 그 중 가장 큰 값을 찾습니다
# len(sublist) for sublist in seq는 seq 리스트의 각 부분 시퀀스(sublist)의 길이를 계산하여 리스트 또는 제너레이터 형태로 반환
'''
max_len = max(len(sublist) for sublist in seq) # seq에 저장된 최대 토큰 길이 찾기
print(max_len)

24


아래 코드는
* seq 리스트의 시퀀스들을 max_len 길이에 맞춰 앞부분에 0으로 패딩 처리하는 함수를 정의하고,
* 이를 사용하여 패딩된 데이터를 생성한 후,
* 첫 번째 패딩된 시퀀스를 출력

이는 모델에 입력되는 시퀀스들의 길이를 통일하기 위해 사용됩니다.

In [25]:
'''
1. 함수 정의 및 설명:
    # pre_zeropadding 함수는 두 개의 인자를 받습니다:
        - seq: 패딩 처리할 시퀀스들의 리스트.
        - max_len: 시퀀스의 목표 길이.
    # 이 함수는 seq 리스트의 각 시퀀스(i)를 max_len 길이에 맞춰 앞부분에 0으로 패딩 처리한 후, 이를 numpy 배열로 반환
    # 리스트 내포(list comprehension)를 사용하여 각 시퀀스에 대해 다음과 같은 처리:
        - i[:max_len]: 시퀀스 i의 길이가 max_len 이상이면 앞부분의 max_len 길이만큼 자릅니다.
        - [0] * (max_len - len(i)) + i: 시퀀스 i의 길이가 max_len보다 짧으면 앞부분에 필요한 만큼의 0을 추가하여 max_len 길이에 맞춥니다.
    # 최종 결과는 numpy 배열로 변환
2. 패딩 처리된 데이터 생성:
    # pre_zeropadding 함수를 호출하여 seq 리스트의 시퀀스들을 max_len 길이에 맞춰 패딩 처리
3. 첫 번째 패딩된 시퀀스 출력:
 # 패딩 처리된 데이터(zero_padding_data)의 첫 번째 시퀀스를 출력
'''
def pre_zeropadding(seq, max_len): # max_len 길이에 맞춰서 0 으로 padding 처리 (앞부분에 padding 처리)
    return np.array([i[:max_len] if len(i) >= max_len else [0] * (max_len - len(i)) + i for i in seq])
zero_padding_data = pre_zeropadding(seq, max_len)
zero_padding_data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3, 273])

아래 코드는
* 패딩 처리된 데이터에서 마지막 열을 레이블(label)로,
* 나머지 부분을 입력 데이터(input_x)로 분리

이를 통해 모델이 입력 데이터를 사용하여 예측을 수행하고, 마지막 단어를 예측하는 문제를 해결할 수 있습니다.

In [29]:
# zero_padding_data의 모든 행에 대해 마지막 열을 제외한 나머지 열을 input_x에 저장(:는 모든 행, :-1는 마지막 열을 제외한 모든 열)
# input_x는 모델의 입력 데이터로 사용됨
# zero_padding_data의 모든 행에 대해 마지막 열을 label에 저장(:는 모든 행, -1는 마지막 열)
# label은 모델의 예측 대상이 되는 레이블로 사용됨
input_x = zero_padding_data[:,:-1]
label = zero_padding_data[:,-1]

In [30]:
input_x[:5] # input_x 리스트의 처음 5개의 요소를 출력

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   3, 273],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   3, 273,  66],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   3, 273,  66,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   3, 273,  66,   1, 467]])

In [32]:
label[:5] # label 리스트의 처음 5개의 요소를 출력

array([ 273,   66,    1,  467, 1582])

## 1-2. Custom Dataset class 구축하기

> 1-1에서 진행한 전처리 진행을 모듈화 시켜서 하나의 class로 구현

### Custom Dataset 정의하기
* 1-1에서 진행한 전처리 과정을 모두 함수화 시켜서 하나의 class로 구축합니다.
* 이로 인해, 손쉬운 모듈화가 가능합니다.
* 데이터를 변환하는 과정은 되도록이면 getitem 이 아닌 init 부분에 하여, 전처리하는 시간을 줄이도록 합니다.
  * init 부분에 한 번에 하게 되면 dataset을 정의할 때만 변환 시간이 소요되고, 그 이후로는 데이터 전처리 시간이 소요되지 않습니다.

* [Custom Dataset 구축 - Pytorch 공식 튜토리얼](https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html)

> PyTorch의 Dataset 클래스를 상속하여 사용자 정의 데이터셋 클래스를 구현한 것

아래 클래스는
* 텍스트 데이터를 토큰화하고,
* 패딩 처리하여
* 입력 데이터와 레이블로 변환하며,
* 이를 PyTorch의 Dataset 형태로 제공.

이를 통해 데이터로더(DataLoader)를 사용하여 모델 훈련에 필요한 배치를 손쉽게 생성할 수 있습니다

In [33]:
# 클래스 정의 및 초기화
class CustomDataset(Dataset):  # Dataset 이라는 모듈을 상속받음
    def __init__(self, data, vocab, tokenizer, max_len): # data, vocab, tokenizer, max_len을 인자로 받아 초기화
        self.data = data
        self.vocab = vocab
        self.max_len = max_len
        self.tokenizer = tokenizer
        # make_sequence 메서드를 호출하여 data를 시퀀스 형태로 변환 (next word prediction을 하기 위한 형태로 변환)
        seq = self.make_sequence(self.data, self.vocab, self.tokenizer)
        # pre_zeropadding 메서드를 호출하여 시퀀스를 패딩 처리 (zero padding으로 채워줌)
        self.seq = self.pre_zeropadding(seq, self.max_len)
        # 패딩 처리된 시퀀스를 torch.tensor로 변환하여 입력 데이터(self.X)와 레이블(self.label)을 생성
        self.X = torch.tensor(self.seq[:,:-1])
        self.label = torch.tensor(self.seq[:,-1])


    # 시퀀스 생성
    '''
    # data의 각 항목에 대해 tokenizer를 사용하여 토큰화한 후, 각 토큰을 vocab을 사용해 인덱스로 변환
    # 각 토큰 리스트에 대해 부분 시퀀스를 생성하여 seq 리스트에 추가
    # seq 리스트를 반환
    '''
    def make_sequence(self, data, vocab, tokenizer):
        seq = []
        for i in data:
            token_id = vocab.lookup_indices(tokenizer(i))
            for j in range(1, len(token_id)):
                sequence = token_id[:j+1]
                seq.append(sequence)
        return seq


    # 패딩 처리 (패딩 처리된 시퀀스를 numpy 배열로 반환)
    def pre_zeropadding(self, seq, max_len): # max_len 길이에 맞춰서 0 으로 padding 처리 (앞부분에 padding 처리)
        return np.array([i[:max_len] if len(i) >= max_len else [0] * (max_len - len(i)) + i for i in seq])

    # dataset의 전체 길이 반환
    '''
        # __len__ 메서드는 데이터셋의 전체 길이를 반환
        # len(self.X)는 데이터셋의 샘플 수를 반환
    '''
    def __len__(self):
        return len(self.X)

    # dataset 접근
    '''
        # __getitem__ 메서드는 주어진 인덱스(idx)에 해당하는 샘플을 반환
        # self.X[idx]는 입력 데이터를 반환하고, self.label[idx]는 해당 레이블을 반환
        # 반환된 값은 모델 훈련 시 사용
    '''
    def __getitem__(self, idx):
        X = self.X[idx]
        label = self.label[idx]
        return X, label

In [36]:
# 텍스트 정리 함수 정의
def cleaning_text(text):
    cleaned_text = re.sub( r"[^a-zA-Z0-9.,@#!\s']+", "", text) # 특수문자 를 모두 지우는 작업을 수행합니다.
    cleaned_text = cleaned_text.replace(u'\xa0',u' ') # No-break space를 unicode 빈칸으로 변환
    cleaned_text = cleaned_text.replace('\u200a',' ') # unicode 빈칸을 빈칸으로 변환
    return cleaned_text

'''
# 텍스트 데이터 전처리
    - map(cleaning_text, data)는 data 리스트의 각 요소에 대해 cleaning_text 함수를 적용
    - list(map(...))는 이 결과를 리스트로 변환하여 data 변수에 저장
    - data 리스트의 모든 텍스트 데이터가 정리된 형태로 변환
# 토크나이저 설정
    - get_tokenizer("basic_english")는 기본 영어 토크나이저를 가져옵니다.
    - 이 토크나이저는 텍스트 데이터를 단어 단위로 분리하는 역할
# 단어 사전 생성
    - data 리스트의 각 요소에 대해 토크나이저를 적용하고, 이를 바탕으로 단어 사전을 생성
    - map(tokenizer, data)는 data 리스트의 각 요소를 토큰화
# 특수 토큰 추가
    - vocab.insert_token('<pad>', 0)는 단어 사전에 '<pad>'라는 특수 토큰을 추가하고, 이 토큰의 인덱스를 0으로 설정
    - '<pad>' 토큰은 일반적으로 시퀀스의 길이를 맞추기 위해 사용
# 최대 시퀀스 길이 설정
    - max_len 변수에 최대 시퀀스 길이로 20을 설정
    - 이는 이후에 시퀀스를 패딩할 때 사용
'''
data = list(map(cleaning_text, data))
tokenizer = get_tokenizer("basic_english")
vocab = torchtext.vocab.build_vocab_from_iterator(map(tokenizer, data))
vocab.insert_token('<pad>',0)
max_len = 20

In [37]:
# train set, validation set, test set으로 data set을 나눕니다. 8 : 1 : 1 의 비율로 나눕니다.
train, test = train_test_split(data, test_size = .2, random_state = 42)
val, test = train_test_split(test, test_size = .5, random_state = 42)

In [38]:
print("Train 개수: ", len(train))
print("Validation 개수: ", len(val))
print("Test 개수: ", len(test))

Train 개수:  5206
Validation 개수:  651
Test 개수:  651


In [39]:
train_dataset = CustomDataset(train, vocab, tokenizer, max_len)
valid_dataset = CustomDataset(val, vocab, tokenizer, max_len)
test_dataset = CustomDataset(test, vocab, tokenizer, max_len)

In [40]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

# 2.Next word prediction 모델 구현

> Next word prediction을 위한 DNN 모델을 직접 구현하고, 학습

## 2-1 Next word prediction을 위한 DNN 모델 구축

> Next word prediction을 위한 DNN 모델을 직접 구축

### Next word prediction을 위한 DNN 모델 구축
* DNN 구현 (2)에서 학습하였던, DNN 모델을 기반에 `nn.Embedding`을 추가하여 next word prediction을 하기 위한 DNN 모델을 구축합니다.
* Embedding이란?
  * 텍스트나 범주형 데이터와 같이 모델이 처리하기 어려운 형태의 데이터를 수치 형태로 변환하는 기술입니다.
  * 주어진 데이터를 저차원의 벡터 공간에 표현하는 방법으로, 단어, 문장, 범주형 변수 등을 고정된 길이의 실수 벡터로 매핑하여 표현합니다.
* `nn.Embedding`
  * num_embedding : embedding할 input값의 수를 의미합니다. 자연어처리에선 단어 사전의 크기와 동일합니다.
  * embedding_dim : embedding 벡터의 차원을 의미합니다.

* [torch.nn.Embedding - Pytorch 공식 튜토리얼](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
* [Embedding 설명](https://wikidocs.net/64779)

> 아래 코드는 단어 예측 모델인 NextWordPredictionModel을 PyTorch로 구현한 것

* NextWordPredictionModel 클래스는 단어 예측을 위한 신경망 모델을 정의.
* 모델은 임베딩 레이어, 여러 은닉층, 마지막으로 클래스를 예측하는 레이어로 구성됨.
* 입력 시퀀스를 임베딩 벡터로 변환하고, 각 단어의 임베딩을 합쳐 문장 단위의 벡터로 만든 후, 이를 통해 다음 단어를 예측함.
* count_parameters 메서드를 통해 모델의 학습 가능한 파라미터 개수를 확인할 수 있음.

In [41]:
# 클래스 정의 및 초기화
'''
# __init__ 메서드는 클래스 초기화 시 호출됩니다.
# vocab_size: 단어 사전의 크기.
# embedding_dims: 임베딩 차원 크기.
# hidden_dims: 은닉층 차원 크기 리스트.
# num_classes: 예측할 클래스(단어)의 수.
# dropout_ratio: 드롭아웃 비율.
# set_super: 부모 클래스 초기화 여부를 결정하는 플래그.

초기화 메서드 내부
    # self.embedding: 단어 임베딩 레이어를 정의. padding_idx=0은 패딩 인덱스가 0임을 의미.
    # self.hidden_dims: 은닉층 차원 크기 리스트를 저장.
    # self.layers: 은닉층을 저장할 모듈 리스트를 정의.
    # for 루프: 각 은닉층을 정의하고 Linear, BatchNorm1d, ReLU, Dropout 레이어를 순서대로 추가.
    # self.classifier: 마지막 은닉층을 통해 클래스(단어) 예측을 위한 선형 레이어를 정의.
    # self.softmax: 출력 값을 확률로 변환하기 위해 LogSoftmax 레이어를 정의.
'''
class NextWordPredictionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dims, hidden_dims, num_classes, dropout_ratio, set_super):
        if set_super:
            super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dims, padding_idx = 0) # padding index 설정 => gradient 계산에서 제외
        self.hidden_dims = hidden_dims
        self.layers = nn.ModuleList()
        self.num_classes = num_classes
        for i in range(len(self.hidden_dims) - 1):
            self.layers.append(nn.Linear(self.hidden_dims[i], self.hidden_dims[i+1]))

            self.layers.append(nn.BatchNorm1d(self.hidden_dims[i+1]))

            self.layers.append(nn.ReLU())

            self.layers.append(nn.Dropout(dropout_ratio))

        self.classifier = nn.Linear(self.hidden_dims[-1], self.num_classes) # 다음에 올 단어가 무엇인지를 맞추기. num_classes = 포켓사이즈
        self.softmax = nn.LogSoftmax(dim = 1)  # 확률로 변환


    # 순전파 메서드
    '''
    # forward 메서드는 모델의 순전파 과정을 정의
    # x: 입력 시퀀스 텐서. 크기는 [batch_size, sequence_len]
    # x = self.embedding(x): 입력 시퀀스를 임베딩 벡터로 변환. 크기는 [batch_size, sequence_len, embedding_dim]
    # x = torch.sum(x, dim=1): 시퀀스의 각 단어의 임베딩을 합쳐서 문장 단위의 임베딩 벡터로 만듦. 크기는 [batch_size, embedding_dim]
    # for layer in self.layers: 정의된 모든 은닉층을 순서대로 통과시킴
    # output = self.classifier(x): 마지막 은닉층의 출력을 통해 클래스를 예측. 크기는 [batch_size, num_classes]
    # output = self.softmax(output): 예측 값을 확률로 변환. 크기는 [batch_size, num_classes]
    # return output: 예측 결과를 반환
    '''
    def forward(self, x):
        '''
        INPUT:
            x: [batch_size, sequence_len] # padding 제외
        OUTPUT:
            output : [batch_size, vocab_size]
        '''
        x = self.embedding(x) # [batch_size, sequence_len, embedding_dim]
        x = torch.sum(x, dim=1) # [batch_size, embedding_dim] 각 문장에 대해 임베딩된 단어들을 합쳐서, 해당 문장에 대한 임베딩 벡터로 만들어줍니다.
        for layer in self.layers:
            x = layer(x)

        output = self.classifier(x) # [batch_size, num_classes]
        output = self.softmax(output) # [batch_size, num_classes]
        return output

    # 파라미터 개수 세기
    '''
    # count_parameters 메서드는 학습 가능한 파라미터의 개수를 반환
    # self.parameters(): 모델의 모든 파라미터를 반환
    # p.numel(): 파라미터 텐서의 요소 개수를 반환
    # if p.requires_grad: 학습 가능한 파라미터만을 선택
    # sum(...): 모든 학습 가능한 파라미터의 개수를 합산하여 반환
    '''
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

## 2-2. 모델 학습 및 추론

> Next word prediction 모델을 직접 학습하고, text를 직접 넣어 next word prediction을 직접 수행

### Next word prediction 학습하기
* DNN 모델을 학습하기 위해 모델의 파라미터를 정해줍니다.
* embedding layer와 fully connected layer의 연산이 가능하게 하기 위해 hidden dimension 리스트 구성 시, embedding dimension을 첫번째 값으로 설정합니다.
* 예측하려는 label의 개수는 단어 사전에 있는 단어의 개수와 동일합니다.

> 아래 코드는 앞의 내용(DNN구현(2))과 100% 동일

In [42]:
# training 코드, evaluation 코드, training loop 코드
def training(model, dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    train_loss = 0.0
    train_accuracy = 0

    tbar = tqdm(dataloader)
    for texts, labels in tbar:
        texts = texts.to(device)
        labels = labels.to(device)

        # 순전파
        outputs = model(texts)

        loss = criterion(outputs, labels)

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 손실과 정확도 계산
        train_loss += loss.item()
        # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
        _, predicted = torch.max(outputs, dim=1)


        train_accuracy += (predicted == labels).sum().item()

        # tqdm의 진행바에 표시될 설명 텍스트를 설정
        tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}")

    # 에폭별 학습 결과 출력
    train_loss = train_loss / len(dataloader)
    train_accuracy = train_accuracy / len(train_dataset)

    return model, train_loss, train_accuracy

def evaluation(model, dataloader, val_dataset, criterion, device, epoch, num_epochs):
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    valid_accuracy = 0

    with torch.no_grad(): # model의 업데이트 막기
        tbar = tqdm(dataloader)
        for texts, labels in tbar:
            texts = texts.to(device)
            labels = labels.to(device)

            # 순전파
            outputs = model(texts)
            loss = criterion(outputs, labels)

            # 손실과 정확도 계산
            valid_loss += loss.item()
            # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
            _, predicted = torch.max(outputs, 1)
            # _, true_labels = torch.max(labels, dim=1)
            valid_accuracy += (predicted == labels).sum().item()


            # tqdm의 진행바에 표시될 설명 텍스트를 설정
            tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Valid Loss: {loss.item():.4f}")

    valid_loss = valid_loss / len(dataloader)
    valid_accuracy = valid_accuracy / len(val_dataset)

    return model, valid_loss, valid_accuracy


def training_loop(model, train_dataloader, valid_dataloader, train_dataset, val_dataset, criterion, optimizer, device, num_epochs, patience, model_name):
    best_valid_loss = float('inf')  # 가장 좋은 validation loss를 저장
    early_stop_counter = 0  # 카운터
    valid_max_accuracy = -1

    for epoch in range(num_epochs):
        model, train_loss, train_accuracy = training(model, train_dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs)
        model, valid_loss, valid_accuracy = evaluation(model, valid_dataloader, val_dataset, criterion, device, epoch, num_epochs)

        if valid_accuracy > valid_max_accuracy:
            valid_max_accuracy = valid_accuracy

        # validation loss가 감소하면 모델 저장 및 카운터 리셋
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"./model_{model_name}.pt")
            early_stop_counter = 0

        # validation loss가 증가하거나 같으면 카운터 증가
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

        # 조기 종료 카운터가 설정한 patience를 초과하면 학습 종료
        if early_stop_counter >= patience:
            print("Early stopping")
            break

    return model, valid_max_accuracy

> 학습 진행

> 아래 코드는 NextWordPredictionModel을 설정하고 학습시키는 과정을 보여줍니다.

* 모델을 설정하고 초기화합니다.
* 학습률, 단어 사전 크기, 임베딩 차원, 은닉층 차원, 드롭아웃 비율 등을 설정합니다.
* 최적화 알고리즘(Adam)과 손실 함수(NLLLoss)를 설정합니다.
* training_loop 함수를 호출하여 모델을 학습하고, 학습된 모델과 검증 데이터셋에서의 최대 정확도를 반환받습니다.
* 검증 데이터셋에서의 최대 정확도를 출력합니다.

In [43]:
# 학습 설정 및 모델 초기화
'''
# lr = 1e-3: 학습률(learning rate)을 0.001로 설정
# vocab_size = len(vocab.get_stoi()): 단어 사전의 크기를 설정
# embedding_dims = 512: 임베딩 차원 크기를 512로 설정
# hidden_dims = [embedding_dims, embedding_dims*4, embedding_dims*2, embedding_dims]: 은닉층의 차원 크기 리스트를 설정 [512, 512*4=2048, 512*2=1024, 512]]

# model = NextWordPredictionModel(...): NextWordPredictionModel의 인스턴스를 생성하고, 필요한 매개변수를 전달하여 초기화
'''
lr = 1e-3
vocab_size = len(vocab.get_stoi())
embedding_dims = 512
hidden_dims = [embedding_dims, embedding_dims*4, embedding_dims*2, embedding_dims]
model = NextWordPredictionModel(vocab_size = vocab_size, embedding_dims = embedding_dims, hidden_dims = hidden_dims, num_classes = vocab_size, \
            dropout_ratio = 0.2, set_super = True).to(device)


# 학습 설정
num_epochs = 100
patience = 3
model_name = 'next'


# 최적화 및 손실 함수 설정
'''
# optimizer = optim.Adam(model.parameters(), lr = lr): Adam 옵티마이저를 설정. 학습률은 0.001
# criterion = nn.NLLLoss(ignore_index=0): 손실 함수를 NLLLoss로 설정. 패딩된 부분(ignore_index=0)은 손실 계산에서 제외
'''
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.NLLLoss(ignore_index=0) # padding 한 부분 제외

# 모델 학습
'''
# training_loop(...): 모델을 학습시키기 위한 함수
    # model: 학습할 모델
    # train_dataloader: 훈련 데이터 로더
    # valid_dataloader: 검증 데이터 로더
    # train_dataset: 훈련 데이터셋
    # valid_dataset: 검증 데이터셋
    # criterion: 손실 함수
    # optimizer: 옵티마이저
    # device: 학습에 사용할 장치
    # num_epochs: 최대 에포크 수
    # patience: 조기 종료를 위한 patience
    # model_name: 모델 이름
# training_loop 함수는 학습된 모델과 검증 데이터셋에서의 최대 정확도를 반환
'''
model, valid_max_accuracy = training_loop(model, train_dataloader, valid_dataloader, train_dataset, valid_dataset, criterion, optimizer, device, num_epochs, patience, model_name)
print('Valid max accuracy : ', valid_max_accuracy)

  0%|          | 0/1159 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

Epoch [1/100], Train Loss: 7.3720, Train Accuracy: 0.0638 Valid Loss: 7.2151, Valid Accuracy: 0.0696


  0%|          | 0/1159 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

...

  0%|          | 0/149 [00:00<?, ?it/s]

Epoch [4/100], Train Loss: 6.0641, Train Accuracy: 0.0935 Valid Loss: 7.4605, Valid Accuracy: 0.0908


  0%|          | 0/1159 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

Epoch [5/100], Train Loss: 5.7944, Train Accuracy: 0.1033 Valid Loss: 7.6211, Valid Accuracy: 0.0948
Early stopping
Valid max accuracy :  0.09476572393414943


### Next word prediction 평가하기
* 학습한 DNN 모델을 accuracy score로 평가합니다.

> 아래 코드는 저장된 모델을 로드하고, 테스트 데이터셋을 사용하여 예측을 수행하며, 그 예측 결과를 바탕으로 모델의 정확도를 계산하는 과정을 보여줍니다.

* 저장된 모델 가중치를 로드하고, 평가 모드로 전환합니다.
* 테스트 데이터셋을 사용하여 예측을 수행하고, 예측 결과와 실제 레이블을 각각 total_preds와 total_labels 리스트에 저장합니다.
* 예측 결과와 실제 레이블을 비교하여 모델의 정확도를 계산하고 출력합니다.

DNN구현(2)의 내용과 거의 동일

In [44]:
# 모델 로드 및 평가 모드 설정
'''
# model.load_state_dict(torch.load("./model_next.pt")): 저장된 모델의 가중치(state dict)를 로드합니다.
# model = model.to(device): 모델을 지정된 장치(GPU 또는 CPU)로 이동합니다.
# model.eval(): 모델을 평가 모드로 전환합니다. 이 모드에서는 드롭아웃 등의 레이어가 비활성화됩니다.
'''
model.load_state_dict(torch.load("./model_next.pt")) # 모델 불러오기
model = model.to(device)
model.eval()


# 초기화 : total_labels와 total_preds 리스트를 초기화. 이 리스트는 모든 예측 값과 실제 레이블을 저장
total_labels = []
total_preds = []


# 예측수행
'''
# with torch.no_grad():
    이 블록 안에서는 그래디언트 계산을 하지 않도록 설정. 이를 통해 메모리 사용량을 줄이고 계산 속도를 높임
# for texts, labels in tqdm(test_dataloader):
    test_dataloader에서 배치를 하나씩 가져옴. tqdm은 진행 상황을 표시.
# texts = texts.to(device): 입력 텍스트 데이터를 장치(GPU 또는 CPU)로 이동.
# labels = labels: 레이블 데이터를 장치로 이동하지 않고 그대로 사용.
# outputs = model(texts): 모델을 사용하여 예측 값을 계산.

# _, predicted = torch.max(outputs.data, 1):
    예측 결과에서 가장 높은 값을 가진 클래스 인덱스를 가져옴. dim=1은 각 샘플에 대해 클래스 차원에서 최대값을 찾는 것을 의미.
# total_preds.extend(predicted.detach().cpu().tolist()):
    예측 값을 total_preds 리스트에 추가. detach().cpu().tolist()는 텐서를 분리하고 CPU로 이동시킨 후 리스트로 변환.
# total_labels.extend(labels.tolist()):
    실제 레이블을 total_labels 리스트에 추가.
'''
with torch.no_grad():
    for texts, labels in tqdm(test_dataloader):
        texts = texts.to(device)
        labels = labels

        outputs = model(texts)
        # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
        _, predicted = torch.max(outputs.data, 1)

        total_preds.extend(predicted.detach().cpu().tolist())
        total_labels.extend(labels.tolist())


# 정확도 계산 및 출력
total_preds = np.array(total_preds)  # numpy 배열로 변환
total_labels = np.array(total_labels)  # numpy 배열로 변환
nwp_dnn_acc = accuracy_score(total_labels, total_preds) # total_labels와 total_preds를 비교하여 정확도 계산
print("Next word prediction DNN model accuracy : ", nwp_dnn_acc)

  0%|          | 0/143 [00:00<?, ?it/s]

Next word prediction DNN model accuracy :  0.07742782152230972


> accuracy 수치가 너무 낮은 것 아닐까?

8618개 중에서 다음에 올 단어를 정확하게 예측하는게 그만큼 어렵다.

In [45]:
print(vocab_size)

8618
